In [4]:
import math
from matplotlib import pyplot as plt
import numpy as np
from cmath import sqrt

# ejercicio 3

In [41]:
def func_fitness_1 (x):
    f_x = -x* math.sin(math.sqrt(abs(x)))
    return f_x

In [52]:
f=func_fitness_1(1)
print(np.random.uniform(-512,512,100))

[-311.4340289    96.74563493  402.86783347  211.70906353  461.97378007
 -285.31540113  503.26608174 -220.02739667   38.91209776 -379.97711681
 -219.706149     52.3770218   192.02024999  252.54289597  439.76497693
 -495.98618142   14.89653061 -258.64058118  -72.19072252   78.13823776
  -42.05373492  254.46968067 -368.08210537  324.91897623  431.72889146
 -135.21176176  -97.08086564   71.78453986 -318.51606275 -128.46547349
 -390.5664368   367.64414418  148.30645557 -266.65293918 -138.21958203
  -77.67497418 -169.66523474  236.23730736  325.20960439  403.7042979
  511.06264091  401.26958464  132.32707096  -90.62578281 -466.96193564
 -158.96108232 -504.69858573 -174.57995262  494.73762664  397.68507659
 -134.53308065   91.58166303  443.73301938  -57.42722804 -226.26281533
  -64.11896878  428.90697633   36.1780696  -279.05546861  -32.55025621
  -86.21413395 -419.91949043  494.65793452  265.50590343  -29.64188447
  -66.09456677  420.77834162   33.17438201   15.89821692 -163.07634466
  301.0

In [124]:
from cmath import isclose
from itertools import count

def alg_enjambre (particulas,f,c1,c2):
    x=np.random.uniform(-512,512,particulas)
    v=0
    yk=x
    yk_global=yk[0]
    # print(f'yk[0] = {func_fitness_1(yk[0])}')
    counter=0
    while (counter!=6):
        # print(counter)
        f_v = []
        for k in range(particulas):
            
            f_v.append(f(x[k]))
           
            if(f(x[k]) < f(yk[k])):
                yk[k]=x[k]
            if(f(yk[k]) < f(yk_global)):
                # print(f'entro')
                yk_global=yk[k]
                counter=0
            # if isclose(yk_global,yk[k]):
            #     counter=0
          
        # print(f_v)
        # print(counter)

        for k in range(particulas):
            r_1 = np.random.rand(x.shape[0])
            r_2 = np.random.rand(x.shape[0])
            v = v + (c1*r_1)*(yk[k]-x[k]) + (c2*r_2)*(yk_global-x[k])
            # x=x+v
        counter= counter + 1
    return yk_global
          
    

In [139]:
winner = alg_enjambre(100,func_fitness_1,1.2,0.9)
print(func_fitness_1(winner))

-418.88026799983317


-418.4954630775152
